# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [54]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df2 = df.copy()
from sklearn.preprocessing import LabelEncoder
for col in df2.columns:
    if df2[col].dtypes == object:
        df2[col]=LabelEncoder().fit_transform(df2[col])
    
df2.dtypes


loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                  int32
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                   int32
debt_settlement_flag            int32
Length: 86, dtype: object

# Split the Data into Training and Testing

In [55]:
# Create our features
X = df2.drop('loan_status', axis=1)

# Create our target
y = df2['loan_status'].ravel()
y = pd.Index(y)

In [56]:
X.describe()

loan_amnt      int_rate   installment  home_ownership    annual_inc  \
count  68817.000000  68817.000000  68817.000000    68817.000000  6.881700e+04   
mean   16677.594562      0.127718    480.652863        1.812779  8.821371e+04   
std    10277.348590      0.048130    288.062432        0.941313  1.155800e+05   
min     1000.000000      0.060000     30.890000        0.000000  4.000000e+01   
25%     9000.000000      0.088100    265.730000        1.000000  5.000000e+04   
50%    15000.000000      0.118000    404.560000        1.000000  7.300000e+04   
75%    24000.000000      0.155700    648.100000        3.000000  1.040000e+05   
max    40000.000000      0.308400   1676.230000        3.000000  8.797500e+06   

       verification_status       issue_d  pymnt_plan           dti  \
count         68817.000000  68817.000000     68817.0  68817.000000   
mean              0.669994      0.805542         0.0     21.778153   
std               0.719105      0.714932         0.0     20.199244   
min               0.000000      0.000000         0.0      0.000000   
25%               0.000000      0.000000         0.0     13.890000   
50%               1.000000      1.000000         0.0     19.760000   
75%               1.000000      1.000000         0.0     26.660000   
max               2.000000      2.000000         0.0    999.000000   

        delinq_2yrs  ...  pct_tl_nvr_dlq  percent_bc_gt_75  \
count  68817.000000  ...    68817.000000      68817.000000   
mean       0.217766  ...       95.057627         30.626217   
std        0.718367  ...        8.326426         33.631463   
min        0.000000  ...       20.000000          0.000000   
25%        0.000000  ...       93.000000          0.000000   
50%        0.000000  ...      100.000000         20.000000   
75%        0.000000  ...      100.000000         50.000000   
max       18.000000  ...      100.000000        100.000000   

       pub_rec_bankruptcies  tax_liens  tot_hi_cred_lim  total_bal_ex_mort  \
count          68817.000000    68817.0     6.881700e+04       6.881700e+04   
mean               0.125972        0.0     2.100332e+05       6.133843e+04   
std                0.336732        0.0     1.928088e+05       5.738798e+04   
min                0.000000        0.0     3.600000e+03       2.350000e+02   
25%                0.000000        0.0     6.697700e+04       2.650300e+04   
50%                0.000000        0.0     1.467100e+05       4.535700e+04   
75%                0.000000        0.0     3.036400e+05       7.657000e+04   
max                4.000000        0.0     3.292782e+06       1.295455e+06   

       total_bc_limit  total_il_high_credit_limit  hardship_flag  \
count    68817.000000                6.881700e+04        68817.0   
mean     29734.128558                5.572240e+04            0.0   
std      26795.394232                5.095845e+04            0.0   
min        100.000000                1.270000e+02            0.0   
25%      11600.000000                2.288000e+04            0.0   
50%      22100.000000                4.200000e+04            0.0   
75%      39300.000000                7.249900e+04            0.0   
max     509400.000000                1.426964e+06            0.0   

       debt_settlement_flag  
count               68817.0  
mean                    0.0  
std                     0.0  
min                     0.0  
25%                     0.0  
50%                     0.0  
75%                     0.0  
max                     0.0  

[8 rows x 85 columns]

In [57]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
dtype: int64

In [58]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Counter(y_train)

Counter({1: 51366, 0: 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [59]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [60]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6594349419740851

In [61]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   72,    29],
       [ 6739, 10365]], dtype=int64)

In [63]:
#The accuracy score is high at around 90%, but this number can be misleading, especially in an unbalanced dataset. Let’s examine the classification report to assess the results further. We’ll use the classification_report_imbalanced to do so.
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.71      0.61      0.02      0.66      0.44       101
          1       1.00      0.61      0.71      0.75      0.66      0.43     17104

avg / total       0.99      0.61      0.71      0.75      0.66      0.43     17205



### SMOTE Oversampling

In [64]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [65]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6591391394752197

In [67]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)


array([[   63,    38],
       [ 5225, 11879]], dtype=int64)

In [68]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.62      0.69      0.02      0.66      0.43       101
          1       1.00      0.69      0.62      0.82      0.66      0.44     17104

avg / total       0.99      0.69      0.62      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [69]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [70]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6591391394752197

In [72]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   67,    34],
       [10331,  6773]], dtype=int64)

In [73]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.66      0.40      0.01      0.51      0.27       101
          1       1.00      0.40      0.66      0.57      0.51      0.26     17104

avg / total       0.99      0.40      0.66      0.56      0.51      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [74]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [75]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [76]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5296777894580852

In [77]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   74,    27],
       [ 7080, 10024]], dtype=int64)

In [78]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.73      0.59      0.02      0.66      0.44       101
          1       1.00      0.59      0.73      0.74      0.66      0.42     17104

avg / total       0.99      0.59      0.73      0.73      0.66      0.42     17205

